# Gun Violence Dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# gun violence data 2013-2018
df_gun_violence = pd.read_csv('https://media.githubusercontent.com/media/Cng000/Gun-violence-analysis/carlos/dataset/gun-violence-data_01-2013_03-2018.csv')

In [ ]:
# mean annual household income (2016)
df_income = pd.read_csv("https://raw.githubusercontent.com/Cng000/Gun-violence-analysis/gun-violence-data/dataset/2016_mean_income_raw_data.csv", skiprows=2)

## Gun Violence Analysis

In [ ]:
df_gun_violence.info()

In [ ]:
df_gun_violence.head()

In [ ]:
df_gun_violence_new = df[['incident_id', 'date', 'state', 'city_or_county', 'n_killed', 'n_injured', 'congressional_district', 'gun_stolen', 'gun_type', 'incident_characteristics', 'n_guns_involved', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_status', 'participant_type', 'state_house_district', 'state_senate_district']]

In [ ]:
df_gun_violence_new.info()

#### Top ten cities/counties by number of incidents

In [ ]:
df_gun_violence_new['city_or_county'].value_counts().head(10)

In [ ]:
# graph results
y = df_gun_violence_new['city_or_county'].value_counts().index
x = df_gun_violence_new['city_or_county'].value_counts().head(10)
x = list(x)

city = []
for i in range(0,10):
    city.append(y[i])

_ = plt.scatter(x=city, y=x)
_ = plt.xticks(rotation=90)
_ = plt.xlabel("city")
_ = plt.ylabel("# of incidents")
plt.show()

## Mean Income 2016 Analysis

In [ ]:
df_income.info()

In [ ]:
df_income.head()

In [ ]:
# dropped US average income and 'Notes'
df_income.drop(df_income.index[0], inplace=True)
df_income.drop(df_income.index[52:], inplace=True)

In [ ]:
df_income

In [ ]:
# renamed columns
# dropped '$' and convert from str to int
df_income = df_income.rename(index=str, columns={"Location":"state", "Median Annual Household Income": "median_annual_household_income_2016"})
df_income['median_annual_household_income_2016'] = pd.to_numeric(df_income['median_annual_household_income_2016'].str[1:], downcast='integer')
df_income

In [ ]:
# merged both dataframe(gun-violence && 2016_income)
df_gun_income = pd.merge(df_gun_violence_new, df_income, on="state", how="outer")
df_gun_income

In [ ]:
# merge incident count and mean income (mean annual income household 2016 only) per state
incident_count_per_state = df_gun_income['state'].value_counts().to_frame().reset_index().rename(index=str, columns={"index":"state", "state":"incident_count"})
state_incidentCount_medianIncome = pd.merge(incident_count_per_state, df_income, on="state", how="outer")
state_incidentCount_medianIncome


In [ ]:
# graphed (median_income && incident_count)
state_incidentCount_medianIncome.plot.scatter('median_annual_household_income_2016', 'incident_count')